## Imports 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect 
from sqlalchemy import create_engine

## Extract CSVs into DataFrames

In [2]:
population_file= "Resources/population_figures_by_country.csv"
population_df= pd.read_csv(population_file)
population_df.head()

,Country,Year_2015,Year_2016
0,Aruba,104341,104822
1,Afghanistan,33736494,34656032
2,Angola,27859305,28813463
3,Albania,2880703,2876101
4,Andorra,78014,77281


In [3]:
happy2015_file= "Resources/2015.csv"
happy2015_df= pd.read_csv(happy2015_file)
happy2015_df.head()

,Country,Happiness_Score,Economy,Family,Health,Freedom
0,Switzerland,7.587,1.39651,1.34951,0.94143,0.66557
1,Iceland,7.561,1.30232,1.40223,0.94784,0.62877
2,Denmark,7.527,1.32548,1.36058,0.87464,0.64938
3,Norway,7.522,1.45900,1.33095,0.88521,0.66973
4,Canada,7.427,1.32629,1.32261,0.90563,0.63297


In [4]:
happy2015_df.columns = [x.lower() for x in list(happy2015_df.columns)]
happy2015_df.head()


,country,happiness_score,economy,family,health,freedom
0,Switzerland,7.587,1.39651,1.34951,0.94143,0.66557
1,Iceland,7.561,1.30232,1.40223,0.94784,0.62877
2,Denmark,7.527,1.32548,1.36058,0.87464,0.64938
3,Norway,7.522,1.45900,1.33095,0.88521,0.66973
4,Canada,7.427,1.32629,1.32261,0.90563,0.63297


In [5]:
happy2016_file= "Resources/2016.csv"
happy2016_df= pd.read_csv(happy2016_file)
happy2016_df.head()

,Country,Happiness_Score,Economy,Family,Health,Freedom
0,Denmark,7.526,1.44178,1.16374,0.79504,0.57941
1,Switzerland,7.509,1.52733,1.14524,0.86303,0.58557
2,Iceland,7.501,1.42666,1.18326,0.86733,0.56624
3,Norway,7.498,1.57744,1.12690,0.79579,0.59609
4,Finland,7.413,1.40598,1.13464,0.81091,0.57104


## Transform Population DataFrame

In [6]:
# Create a filtered dataframe from specific columns
population_cols = ["Country", "Year_2015", "Year_2016"]
population_transformed= population_df[population_cols].copy()

# Rename the column headers
population_transformed = population_transformed.rename(columns={"Country": "Country",
                                                          "Year_2015": "2015_Population",
                                                          "Year_2016": "2016_Population"})



# Clean the data by dropping duplicates and setting the index
population_transformed.drop_duplicates
#population_transformed.set_index("Country", inplace=True)

population_transformed.head()

,Country,2015_Population,2016_Population
0,Aruba,104341,104822
1,Afghanistan,33736494,34656032
2,Angola,27859305,28813463
3,Albania,2880703,2876101
4,Andorra,78014,77281


## Transform 2015 Happiness DataFrame

In [7]:
# Create a filtered dataframe from specific columns
happy2015_cols = ["country", "economy", "family","freedom","happiness_score", "health"]
happy2015_transformed= happy2015_df[happy2015_cols].copy()

# Clean the data by dropping duplicates and setting the index
happy2015_transformed.drop_duplicates
#happy2015_transformed.set_index("Country", inplace=True)

happy2015_transformed.head()

,country,economy,family,freedom,happiness_score,health
0,Switzerland,1.39651,1.34951,0.66557,7.587,0.94143
1,Iceland,1.30232,1.40223,0.62877,7.561,0.94784
2,Denmark,1.32548,1.36058,0.64938,7.527,0.87464
3,Norway,1.45900,1.33095,0.66973,7.522,0.88521
4,Canada,1.32629,1.32261,0.63297,7.427,0.90563


## Transform 2016 Happiness DataFrame

In [8]:
# Create a filtered dataframe from specific columns
happy2016_cols = ["Country", "Economy", "Family","Freedom","Happiness_Score", "Health"]
happy2016_transformed= happy2016_df[happy2016_cols].copy()

# Clean the data by dropping duplicates and setting the index
happy2016_transformed.drop_duplicates
#happy2016_transformed.set_index("Country", inplace=True)
happy2016_transformed.head()

,Country,Economy,Family,Freedom,Happiness_Score,Health
0,Denmark,1.44178,1.16374,0.57941,7.526,0.79504
1,Switzerland,1.52733,1.14524,0.58557,7.509,0.86303
2,Iceland,1.42666,1.18326,0.56624,7.501,0.86733
3,Norway,1.57744,1.12690,0.59609,7.498,0.79579
4,Finland,1.40598,1.13464,0.57104,7.413,0.81091


## Create database connection

In [9]:
connection_string = "postgres:postgres@localhost:5432/population_happiness"
engine = create_engine(f'postgresql://{connection_string}')

inspector=inspect(engine)

## Check for table

In [10]:
inspector.get_table_names()

['happy_2015', 'happy_2016', 'population']

In [11]:
inspector.get_columns('happy_2015')

[{'name': 'country',
  'type': VARCHAR(length=40),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'family',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'economy',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'health',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'freedom',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'happiness_score',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

## Load DataFrames into database


In [14]:
happy2015_transformed.to_sql(name='happy_2015', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "happy_2015_pkey"
DETAIL:  Key (country)=(Switzerland) already exists.

[SQL: INSERT INTO happy_2015 (country, economy, family, freedom, happiness_score, health) VALUES (%(country)s, %(economy)s, %(family)s, %(freedom)s, %(happiness_score)s, %(health)s)]
[parameters: ({'country': 'Switzerland', 'economy': 1.3965100000000001, 'family': 1.34951, 'freedom': 0.66557, 'happiness_score': 7.587000000000001, 'health': 0.9414299999999999}, {'country': 'Iceland', 'economy': 1.3023200000000001, 'family': 1.4022299999999999, 'freedom': 0.6287699999999999, 'happiness_score': 7.561, 'health': 0.94784}, {'country': 'Denmark', 'economy': 1.32548, 'family': 1.36058, 'freedom': 0.6493800000000001, 'happiness_score': 7.527, 'health': 0.87464}, {'country': 'Norway', 'economy': 1.459, 'family': 1.33095, 'freedom': 0.66973, 'happiness_score': 7.522, 'health': 0.88521}, {'country': 'Canada', 'economy': 1.32629, '

In [15]:
happy2016_transformed.to_sql(name='happy_2016', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "index" of relation "happy_2016" does not exist
LINE 1: INSERT INTO happy_2016 (index, "Country", "Economy", "Family...
                                ^

[SQL: INSERT INTO happy_2016 (index, "Country", "Economy", "Family", "Freedom", "Happiness_Score", "Health") VALUES (%(index)s, %(Country)s, %(Economy)s, %(Family)s, %(Freedom)s, %(Happiness_Score)s, %(Health)s)]
[parameters: ({'index': 0, 'Country': 'Denmark', 'Economy': 1.4417799999999998, 'Family': 1.16374, 'Freedom': 0.5794100000000001, 'Happiness_Score': 7.526, 'Health': 0.79504}, {'index': 1, 'Country': 'Switzerland', 'Economy': 1.5273299999999999, 'Family': 1.14524, 'Freedom': 0.58557, 'Happiness_Score': 7.5089999999999995, 'Health': 0.86303}, {'index': 2, 'Country': 'Iceland', 'Economy': 1.42666, 'Family': 1.18326, 'Freedom': 0.56624, 'Happiness_Score': 7.501, 'Health': 0.8673299999999999}, {'index': 3, 'Country': 'Norway', 'Economy': 1.57744, 'Family': 1.1269, 'Freed

In [ ]:
population_transformed.to_sql(name='population', con=engine, if_exists='append', index=True)

## Confirm data has been added by querying the data

In [ ]:
pd.read_sql_query('select * from happy_2015', con=engine).head()

In [ ]:
pd.read_sql_query('select * from happy_2016', con=engine).head()

In [ ]:
pd.read_sql_query('select * from population', con=engine).head()